# Question 1: Use yfinance to Extract Stock Data (Tesla)

In [ ]:

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup

# QA: Create Ticker object for Tesla
tesla = yf.Ticker("TSLA")

# QB: Extract stock info with history, period max
tesla_data = tesla.history(period="max")

# QC: Reset index
tesla_data.reset_index(inplace=True)

# Display first 5 rows
tesla_data.head()


# Question 2: Webscraping Tesla Revenue Data

In [ ]:

# QA: Download the webpage
url_tesla = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url_tesla).text

# QB: Parse with BeautifulSoup
soup = BeautifulSoup(html_data, "html.parser")

# QC: Extract Tesla Revenue table
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in soup.find_all("tbody")[1].find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
    tesla_revenue = pd.concat(
        [tesla_revenue, pd.DataFrame({"Date":[date], "Revenue":[revenue]})],
        ignore_index=True
    )

# QD: Remove $ and commas
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(",", "")
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace("$", "")

# QE: Remove null / empty
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]

# QF: Display last 5 rows
tesla_revenue.tail()


# Question 3: Use yfinance to Extract Stock Data (GameStop)

In [ ]:

# QA: Create Ticker object for GameStop
gme = yf.Ticker("GME")

# QB: Extract stock info
gme_data = gme.history(period="max")

# QC: Reset index
gme_data.reset_index(inplace=True)

# Display first 5 rows
gme_data.head()


# Question 4: Webscraping GME Revenue Data

In [ ]:

# QA: Download the webpage
url_gme = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data_2 = requests.get(url_gme).text

# QB: Parse HTML
soup = BeautifulSoup(html_data_2, "html.parser")

# QC: Extract GameStop Revenue table
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in soup.find_all("tbody")[1].find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text
    gme_revenue = pd.concat(
        [gme_revenue, pd.DataFrame({"Date":[date], "Revenue":[revenue]})],
        ignore_index=True
    )

# Clean revenue column
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(",", "")
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace("$", "")
gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]

# QD: Display last 5 rows
gme_revenue.tail()


# Question 5 & 6: Plot Tesla and GameStop Stock Graphs

In [ ]:

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

# Tesla graph
make_graph(tesla_data, tesla_revenue, "Tesla")

# GameStop graph
make_graph(gme_data, gme_revenue, "GameStop")
